In [1]:
import numpy as np
import re
from nltk.corpus import stopwords 

In [2]:
stop=stopwords.words('english')

In [3]:

def tokenizer(text):
    text=re.sub('<[^>]*>','',text)
    emoticons=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text= (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-',''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [4]:
def stream_docs(path):
    with open (path,'r',encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text,label = line[:-3],int(line[-2])
            yield text,label

In [5]:
next(stream_docs('movie_data.csv'))

('11841,"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and 

In [6]:
def get_minibatch(doc_stream,size):
    docs,y=[],[]
    try:
        for _ in range(size):
            text,label=next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None,None
    return docs,y

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer

In [8]:
from sklearn.linear_model import SGDClassifier

In [9]:
vect=HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)

In [10]:
clf=SGDClassifier(loss='log',random_state=1,max_iter=1)

In [11]:
doc_stream=stream_docs(path='movie_data.csv')

In [12]:
import pyprind

In [13]:
pbar=pyprind.ProgBar(45)

In [14]:
classes=np.array([0,1])

In [15]:
for _ in range(45):
    x_train,y_train=get_minibatch(doc_stream,size=1000)
    if not x_train:
        break
    x_train=vect.transform(x_train)
    clf.partial_fit(x_train,y_train,classes=classes)
    pbar.update()
            

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:45


In [16]:
x_test,y_test=get_minibatch(doc_stream,size=5000)

In [17]:
x_test = vect.transform(x_test)
    
    

In [18]:
clf.score(x_test,y_test)

0.86719999999999997

In [19]:
clf.partial_fit(x_test,y_test)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=1, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=1, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [20]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=3)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=3)

In [21]:
dest

'movieclassifier\\pkl_objects'

In [22]:
os.getcwd()

'C:\\Users\\Amrit Raj\\Documents\\GitHub'

In [23]:

import os
os.chdir('movieclassifier')

In [25]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [26]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ['great movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))

Prediction: positive
Probability: 95.54%


In [27]:
import sqlite3
import os

In [28]:
if os.path.exists('reviews.sqlite'):
    os.remove('reviews.sqlite')

In [29]:
conn=sqlite3.connect('reviews.sqlite')

In [30]:
c=conn.cursor()

In [32]:
c.execute('CREATE TABLE review_db' '(review TEXT,sentiment INTEGER,date TEXT)')

In [33]:
example1='I Love this movie'

In [36]:
c.execute("INSERT INTO review_db" "(review,sentiment,date) VALUES" "(?,?, DATETIME('now'))",(example1,1))

In [37]:
example2='I dislike this movie'

In [39]:
c.execute("INSERT INTO review_db" "(review,sentiment,date)VALUES" "(?,?,DATETIME('now'))",(example2,0))

In [40]:
conn.commit()

In [41]:
conn.close()

In [42]:
conn=sqlite3.connect('reviews.sqlite')

In [43]:
c=conn.cursor()

In [45]:
c.execute("SELECT * FROM review_db WHERE date" " BETWEEN '2018-01-01 00:00:00' AND DATETIME ('now')")

In [46]:
results=c.fetchall()

In [47]:
conn.close()

In [48]:
results

[('I Love this movie', 1, '2018-02-03 04:14:34'),
 ('I dislike this movie', 0, '2018-02-03 04:16:44')]